In [1]:
import sys
import os

# Set the project root directory
project_root = os.path.abspath("..")   # one level above /notebooks
sys.path.append(project_root)

**Imports**

In [ ]:
import numpy as np
import random

import optuna
import torch
import torch.nn as nn 
import torch.optim as optim 

from src.train_utils import build_model, build_trainer, build_optim, calc_class_weights
from src.dataset import get_datasets, create_data_loaders 

**Setting Seed**

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

train_set, val_set, test_set, classes = get_datasets("../data/PlantVillage",val_split=0.1,test_split=0.1)
num_classes = len(classes)

class_weights = calc_class_weights(train_set, DEVICE) # These weights are to be passed into the loss function later 


In [102]:
print(f"{'-'*10} Class Weights {'-'*10}")
for target, weight in enumerate(class_weights):

    print(f"Class {target}: {weight:.3f}")

---------- Class Weights ----------
Class 0: 0.802
Class 1: 0.541
Class 2: 0.799
Class 3: 0.799
Class 4: 5.240
Class 5: 0.376
Class 6: 0.799
Class 7: 0.419
Class 8: 0.839
Class 9: 0.451
Class 10: 0.477
Class 11: 0.569
Class 12: 0.249
Class 13: 2.138
Class 14: 0.502


# **HyperParam Tuning**

In [ ]:
EPOCHS = 25

def objective(trial: optuna.Trial):

    # Model HyperParams  
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_int("batch_size", 32, 256, step=16)
    dropout = trial.suggest_float("dropout", 0.0, 0.7, step=0.1)

    
    # Other Params
    early_stopper_patience = 10
    delta = 1e-4
    lr_patience = early_stopper_patience // 2
    lr_factor = 0.75
    min_lr = 1e-6
    
    # Dataloaders
    train_loader, val_loader, _  = create_data_loaders(train_dataset=train_set,
                                                       val_dataset=val_set,
                                                       test_dataset=None,
                                                       batch_size=batch_size)
    # Model
    model = build_model(num_classes=num_classes, dropout=dropout, freeze_base=True,
                        device=DEVICE)
    
    # Loss, Optimizer
    loss_fn = nn.CrossEntropyLoss(weight=class_weights) 
    optim = build_optim(model=model, lr=lr, weight_decay=weight_decay)

                                  
    # Trainer
    trainer = build_trainer(model, loss_fn, optim,
                            early_stopper_patience, delta, False, None,
                            lr_factor, lr_patience, min_lr,
                            DEVICE)

    # Train Model
    trainer.train_val_model(epochs=EPOCHS,
                            train_loader=train_loader,
                            val_loader=val_loader)
    
    val_f1 = max(trainer.history['val_f1s'])

    return val_f1

In [ ]:
study = optuna.create_study(
    study_name="plant_village_study",
    direction="maximize",    # because we want to maximise the f1 score 
    storage="sqlite:///plant_village_study.db",
    load_if_exists=True)  

study.optimize(objective, n_trials=50)

print(f"Best trial: {study.best_trial.number}")
print(f"Best val f1-macro: {study.best_value}")
print(f"Best hyperparameters: {study.best_params}")

# **ReTrain the Model Using Best HyperParam Combinations**

In [ ]:
study = optuna.load_study(study_name="plant_village_study", storage="sqlite:///plant_village_study.db")
print(study.best_params)

### **Training Phase 1 - Frozen BackBone**

### **Training Phase 2 - Unfrozen**

In [ ]:
# # Load Best Model
# best_model_checkpoint = 
# new_best_model_checkpoint = 

# model = build_model(num_classes=num_classes, dropout=, freeze_base=False)
# model.load_state_dict(torch.load(best_model_checkpoint['model_state_dict']))


# # Lower lr
# lr = 


# # Data Loaders
# train_loader, val_loader, _  = create_data_loaders(train_dataset=train_set,
#                                                     val_dataset=val_set,
#                                                     test_dataset=None,
#                                                     batch_size=batch_size)


# # Loss, Optimizer
# loss_fn = nn.CrossEntropyLoss()
# optim = build_optim(model=model, lr=lr, weight_decay=weight_decay)

                               
# # Trainer
# trainer = build_trainer(model, loss_fn, optim,
#                         early_stopper_patience, delta, checkpoint_path,
#                         lr_factor, lr_patience, min_lr,
#                         DEVICE)

# # Train Model
# trainer.train_val_model(epochs=EPOCHS,
#                         train_loader=train_loader,
#                         val_loader=val_loader)


In [46]:
# Plot Metric Curves 

In [ ]:
# _, _, test_loader  = create_data_loaders(train_dataset=None, val_dataset=None,
#                                          test_dataset=test_set)

In [ ]:
# def are_dataloaders_identical(dl1, dl2):
#     # Check if lengths are the same
#     if len(dl1) != len(dl2):
#         return False
    
#     # Iterate through both dataloaders simultaneously
#     for batch1, batch2 in zip(dl1, dl2):
#         # Assuming batches are tensors or tuples of tensors
#         if isinstance(batch1, torch.Tensor):
#             if not torch.equal(batch1, batch2):
#                 return False
#         elif isinstance(batch1, (list, tuple)):
#             for t1, t2 in zip(batch1, batch2):
#                 if not torch.equal(t1, t2):
#                     return False
#         else:
#             # Handle other data types in batches as needed
#             if batch1 != batch2:
#                 return False
#     return True